In [ ]:

#scATAC-seq processing after cistopic
import scanpy as sc
import anndata as an
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scanorama
import bbknn
sc.set_figure_params(facecolor="white", figsize=(9, 5),fontsize = 20)

adata_atac_ctrl = sc.read_csv("./Cistopic_ATAC_CTRL.csv")
adata_atac_ars = sc.read_csv("./Cistopic_ATAC_ARS.csv")
adata_atac_ctrl.var_names_make_unique()
adata_atac_ars.var_names_make_unique()


for adata2 in [
    adata_atac_ctrl,
    adata_atac_ars,
]:
    sc.pp.normalize_total(adata2, inplace=True)
    sc.pp.log1p(adata2)
    sc.pp.highly_variable_genes(adata2, flavor="seurat", n_top_genes=2000, inplace=True)
adatas2 = [adata_atac_ars, adata_atac_ctrl]
adatas_cor2 = scanorama.correct_scanpy(adatas2, return_dimred=True)
adata2 = adatas_cor2[0].concatenate(
    adatas_cor2[1],
    batch_key="library_id",
    uns_merge="unique",
    batch_categories=["Control","Arsenic"],
)

sc.tl.pca(adata2)
sc.external.pp.bbknn(adata_spatial2,batch_key="library_id")
sc.tl.umap(adata2)
sc.tl.leiden(adata2, key_added="clusters")
sc.pl.umap(adata2, color=["clusters", "library_id"], palette=sc.pl.palettes.default_20)
plt.tight_layout()
sc.pl.umap(adata2, color=["clusters", "library_id"], palette=sc.pl.palettes.default_20, legend_loc = 'on data',legend_fontsize = 12,legend_fontoutline=2)
plt.tight_layout()
sc.pl.umap(adata2, color=[  "clusters", "library_id"],add_outline=True,size=20, legend_loc = 'on data',frameon=False,save='_atac_cluster.pdf')

adata.write_h5ad('dmerged_ATAC.h5ad')